In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
from statistics import mean
from datetime import timedelta

import math

import warnings  
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("~/Desktop/NCSA_genomics/Python - notebooks/TSForecasting/Data/consolidatedDataForPackage.csv")
data['Display Time'] = pd.to_datetime(data['Display Time'])
data['GlucoseValue'] = pd.to_numeric(data['GlucoseValue'])

In [3]:
def fullDay(data):
        
    dates = list()
    data = data.reset_index(drop=True)
    for i in range(0,len(data.index)):
        dates.append(data['Display Time'][i].date())
    data['Dates'] = dates
    end = data['Dates'].iloc[-1]
    start = data['Dates'].iloc[0]

    indexVals = data[ data['Dates'] == start ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    indexVals = data[ data['Dates'] == end ].index
    # indexVals
    data.drop(indexVals , inplace=True)

    data = data.reset_index(drop=True)

    data.drop(['Dates'], axis=1, inplace=True)

    return data

In [7]:
def hourlySamples(df):
    groupkey = df['Display Time'].values.astype('datetime64[h]')
    result = df.groupby(groupkey).mean().first()
    result = result.reset_index(drop=True)
    return (result)

In [29]:
def adrr(xx, unit):
    
    if unit == 'mg':
        f_bg = 1.509*(np.log(xx['GlucoseValue'])**1.084)-5.381
        xx['F(BG)'] = f_bg
    elif unit == 'mmol':
        f_bg = 1.509*(np.log(xx['GlucoseValue']*18)**1.084)-5.381
        xx['F(BG)'] = f_bg
    else:
        print('Unit should either be mg or mmol')
        return 0
    
    dates = []
    for i in range(len(xx.index)):
        dates.append(xx['Display Time'][i].date())
    xx['Date'] = dates 
    

    for Date, df in xx.groupby('Date'):
        r_BG = 0
        rl_BG = [0]
        rh_BG = [0]
        LR = 0
        HR = 0
        ADDR_daily = []
        for f_BG in df['F(BG)']:
            if f_BG < 0:
                rl_BG.append(f_BG)
            else:
                rh_BG.append(f_BG)

        LR = max(rl_BG)
        HR = max(rh_BG)
        ADDR_daily.append(LR+HR)
    
    
    return round(mean(ADDR_daily),3)
        
            

In [30]:
for subjectId, df in data.groupby('subjectId'):
    df = fullDay(df)
    adrr_value = adrr(df, 'mg')
    print(subjectId,adrr_value)

1636-69-001 4.038
1636-69-001-2 3.455
1636-69-026 3.455
1636-69-028 2.859
1636-69-032 3.174
1636-69-035 2.95
1636-69-048 2.89
1636-69-053 3.409
1636-69-060 3.618
1636-69-064 3.385
1636-69-069 3.455
1636-69-090 3.649
1636-69-091 3.362
1636-69-100 2.979
1636-69-104 3.12
1636-69-104-2 2.905
1636-69-107 3.576
1636-69-111 3.023
1636-69-114 3.238
1636-69-123 2.843
1636-70-1002 3.106
1636-70-1003 2.843
1636-70-1005 3.961
1636-70-1008 3.226
1636-70-1010 3.68
2133-001 3.68
2133-002 3.432
2133-003 3.301
2133-004 3.618
2133-006 2.874
2133-007 2.979
2133-008 3.106
2133-009 3.079
2133-010 3.037
2133-011 3.264
2133-012 3.079
2133-013 3.872
2133-015 3.362
2133-017 3.134
2133-018 4.222
2133-019 2.89
2133-019-2 3.489
2133-020 2.979
2133-021 3.987
2133-022 3.554
2133-023 3.023
2133-024 3.397
2133-025 2.697
2133-026 3.079
2133-027 2.843
2133-028 2.645
2133-030 2.935
2133-032 3.106
2133-033 3.251
2133-035 3.289
2133-036 3.944
2133-037 3.313
2133-039 3.608
2133-040 3.187
2133-041 3.12
GVP01 4.704
GVP03 4.1